# Données WFP - Admin 1

In [2]:
import sys
import os

# Détection automatique de la racine du projet
# On remonte d'un niveau depuis le dossier 'notebooks'
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))

# Ajout au path pour pouvoir faire "from src..."
if project_root not in sys.path:
    sys.path.append(project_root)

# Définition des chemins absolus
DATA_RAW = os.path.join(project_root, "data", "raw")
DATA_PROCESSED = os.path.join(project_root, "data", "processed")
GEO_PATH = os.path.join(project_root, "geo")

print(f"Système configuré. Racine : {project_root}")

Système configuré. Racine : c:\Users\valen\OneDrive\Documents\GitHub\Projet-de-Statapp---WFP-


In [6]:
import pandas as pd
import os

# ==============================================================================
# 1. CONFIGURATION DES CHEMINS
# ==============================================================================
# Ton fichier météo (généré à l'étape précédente)
fldas_path = "../data/raw/FLDAS_Nigeria_Admin2_Aggregated.csv" 
# Le fichier WFP 
wfp_path = "../data/raw/WFP_food_security_data_nigeria.csv"
# Fichier de sortie
output_path = "../data/processed/NIGERIA_WFP_ADMIN1.csv"


In [7]:
# ==============================================================================
# 2. TRAITEMENT CLIMAT (ADMIN 2 -> ADMIN 1)
# ==============================================================================
print("1. Chargement et agrégation des données climatiques...")
df_clim = pd.read_csv(fldas_path)
df_clim['time'] = pd.to_datetime(df_clim['time'])

#On regroupe par DATE et par ÉTAT (NAME_1)
# On fait la moyenne de toutes les LGA (NAME_2) qui sont dans cet état
df_clim_admin1 = df_clim.groupby(['time', 'NAME_1']).mean(numeric_only=True).reset_index()


# 2. CORRECTION DES NOMS 
df_clim_admin1['NAME_1'] = df_clim_admin1['NAME_1'].replace({
    'FederalCapitalTerritory': 'Abuja',  # Le plus important
    'AkwaIbom': 'Akwa Ibom',             # Manque l'espace
    'CrossRiver': 'Cross River',         # Manque l'espace
    'Nasarawa': 'Nassarawa',             # Orthographe avec 2 's' souvent utilisée dans WFP
    'Nassarawa': 'Nassarawa'             # Sécurité si déjà avec 2 s
})

print(f"   -> Données climatiques agrégées au niveau État : {df_clim_admin1.shape}")


1. Chargement et agrégation des données climatiques...
   -> Données climatiques agrégées au niveau État : (2664, 28)


In [8]:

# ==============================================================================
# 3. TRAITEMENT WFP (CIBLE)
# ==============================================================================
print("2. Préparation des données WFP...")
df_wfp = pd.read_csv(wfp_path)

# Création d'une vraie colonne Date (1er du mois)
# On prend l'année du 'quarter' (ex: "2023Q1" -> 2023) et le mois
df_wfp['year'] = df_wfp['quarter'].str[:4].astype(int)
df_wfp['time'] = pd.to_datetime(df_wfp[['year', 'month']].assign(day=1))

# On renomme pour avoir la même clé de jointure
df_wfp = df_wfp.rename(columns={'ADM1_NAME': 'NAME_1'})

# On garde les colonnes utiles
cols_wfp = ['time', 'NAME_1', 'inadequate']
df_wfp_clean = df_wfp[cols_wfp]

print(f"   -> Données WFP prêtes : {df_wfp_clean.shape}")


2. Préparation des données WFP...
   -> Données WFP prêtes : (1010, 3)


In [9]:

# ==============================================================================
# 4. FUSION (MERGE)
# ==============================================================================
print("3. Fusion des deux fichiers...")

# Inner join : on garde uniquement les lignes où on a LA MÉTÉO ET LA CIBLE
df_final = pd.merge(df_wfp_clean, df_clim_admin1, on=['time', 'NAME_1'], how='inner')

# Sauvegarde
df_final.to_csv(output_path, index=False)

print("-" * 30)
print(f"✅ SUCCÈS ! Fichier fusionné sauvegardé : {output_path}")
print(f"Dimensions finales : {df_final.shape}")
print(f"Période couverte : {df_final['time'].min().date()} au {df_final['time'].max().date()}")
print("-" * 30)

# Aperçu
display(df_final.tail())

3. Fusion des deux fichiers...
------------------------------
✅ SUCCÈS ! Fichier fusionné sauvegardé : ../data/processed/NIGERIA_WFP_ADMIN1.csv
Dimensions finales : (973, 29)
Période couverte : 2023-01-01 au 2025-12-01
------------------------------


,time,NAME_1,inadequate,bnds,Y,X,Evap_tavg,LWdown_f_tavg,Lwnet_tavg,Psurf_f_tavg,...,Tair_f_tavg,Wind_f_tavg,SoilMoi00_10cm_tavg,SoilMoi10_40cm_tavg,SoilMoi40_100cm_tavg,SoilMoi100_200cm_tavg,SoilTemp00_10cm_tavg,SoilTemp10_40cm_tavg,SoilTemp40_100cm_tavg,SoilTemp100_200cm_tavg
968,2025-12-01,Rivers,0.160714,0.5,4.823889,6.987234,0.000034,411.810330,-43.801556,100792.671190,...,299.888003,2.907054,0.405510,0.406853,0.417039,0.441677,299.747586,299.546677,299.288960,298.954850
969,2025-12-01,Sokoto,0.456522,0.5,13.043215,5.325762,0.000008,335.779078,-115.339913,97790.403591,...,299.638528,6.335776,0.162425,0.324289,0.333883,0.367944,299.868678,299.929751,300.027598,300.237000
970,2025-12-01,Taraba,0.171429,0.5,8.169318,10.847254,0.000013,357.799072,-108.668333,96841.153500,...,301.657993,4.059479,0.196618,0.341409,0.351616,0.379171,302.027061,301.093865,300.262063,299.349812
971,2025-12-01,Yobe,0.736111,0.5,12.280427,11.269654,0.000007,326.325605,-121.448792,96968.869706,...,298.597382,6.557438,0.148997,0.309747,0.319463,0.352666,299.414977,299.629153,299.892923,300.272008
972,2025-12-01,Zamfara,0.326923,0.5,12.281723,6.331793,0.000011,335.686780,-117.691993,96429.503643,...,299.681968,6.764253,0.150952,0.312316,0.324905,0.367590,300.202766,300.077284,299.990443,299.937183
